<a href="https://colab.research.google.com/github/bulnuo/notebook/blob/main/OpenAI_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Generating SQL Queries with OpenAI GPT-3**

---

This notebook illustrates one of several usecases for Natural Language Processing (NLP) enabled by OpenAI GPT-3 engine ([here](https://openai.com/api/))  - Generating SQL statements from Text.

The NLP engine takes two inputs:
1. A model that describes tables of interest
2. A high-level sentence referring to the model

The output is a SQL query that reflects logic of the sentence.

We will use default NuoDB *Hockey* sample database to experiment with the engine.

First several sections below set-up the environment. 
We then describe the *Hockey* model and generate SQL statements. To check the correctness of the statements we run them against an external NuoDB dtatabase.

So, let's start....



**Section 1 - Installation**

Next 2 steps install openAI package and import it into the Notebook.

First, copy this Notebook into your working space. For that click the blue"Open in Colab" button in the top left corner (if you are in GitHub environment)

Then follow to the next 2 *Cells* in the Notebook and click the *Play* button in square brackets to execute next steps steps. Wait until fist step completes before executing the second one. Note that the second step does output anything if succeeds.

In [ ]:
!pip install --upgrade openai

In [ ]:
import os
import openai

**Section 2 - Getting a Secret Key**

OpenAI is free to users while it is in Beta. But you have to login to get a Secret Key in order to use the service. Follow this link to get the key -  https://beta.openai.com/account/api-keys - it is really easy.

Once you have the key, copy it into the value of *secret_key* variable below.

In [ ]:
secret_key ="paste your secret key here"

**Section 3 - Creating a Model**

A model is a list of table and column names. 
It may look like this for the *Hockey* database:

In [ ]:
model = ["Players(playerID, firstName, lastName, bithYear, birthCountry)", 
         "Teams(year, teamID, name, games, wins, losses)",
         "Scoring(playerID, year, teamID, gamesPlayed, goals, assists)"]


**Section 4 - Defining "Build_Query" Function**

This section is optional for you to review. It constructs the OpenAI request and gets back the results - SQL Query.

You can click on *Show Code* link below to see the implementation

In [ ]:
#@title Implementation

def build_query(secret_key, model, question):

  openai.api_key = secret_key
   
  request="### Postgres SQL tables, with their properties:\n#"
  for i in model:
    request=request+"\n# "+i
  request=request+"\n### "+question
  request=request+"\nSELECT"

  #print(request)

  response = openai.Completion.create(
    model="code-davinci-002",
    prompt=request,
    temperature=0,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["#", ";"]
  )
  
  return "SELECT "+response.choices[0]["text"]



**Experiment 1**

Now we are getting to the interesting part!!!

Let's specify a request below in English and see how it is translated to SQL. The question for the NLP engine is this:

`"A query to find the highest scoring player"`

In [ ]:
question="A query to find the highest scoring player"

query = build_query(secret_key, model, question)
print(query)

SELECT  firstName, lastName, goals, assists
FROM Players, Scoring
WHERE Players.playerID = Scoring.playerID
ORDER BY goals DESC, assists DESC
LIMIT 1


A few observations about the generated query:
1. The engine interpreted "*highest scoring*" as a combination of "*goals*" and "*assists*". We never specified "goals" and "assists" in our question 
2. It selects the highest scoring player by "*goals*" with a tie-breaker of "*assist*"
3. It selects a strategy of using "LIMIT 1" with ordering for a simple implementation of "highest" 

This is very impressive!!

Now let's execute this SQL against a NuODB Hockey database. Here, the output:
```
 FIRSTNAME  LASTNAME  GOALS  ASSISTS  
 ---------- --------- ------ -------- 

   Wayne     Gretzky    92     120
```



**Experiment 2**

Now let's make the same request a bit more complex. Can the engine incorporate statistical functions? For example a statement like this:
```
"A query to list players with highest average goal count"
```



In [ ]:
question="A query to list players with highest average goal count"

query = build_query(secret_key, model, question)
print(query)

SELECT  firstName, lastName, AVG(goals) AS avg_goals
FROM Players
JOIN Scoring ON Players.playerID = Scoring.playerID
GROUP BY firstName, lastName
ORDER BY avg_goals DESC
LIMIT 10







Executing this query results in the folowing output:
```
 FIRSTNAME  LASTNAME      AVG_GOALS     
 ---------- --------- ----------------- 

   Mike     Bossy     57.3              
   Alex     Ovechkin  48.42857142857143 
   Steven   Stamkos   44.75             
   Wayne    Gretzky   40.8695652173913  
   Mario    Lemieux   40.58823529411764 
   Real     Cloutier  39                
   Marcel   Dionne    38.47368421052632 
   Bobby    Hull      38.04166666666666 
   Phil     Esposito  37.73684210526316 
   Anders   Hedberg   37.09090909090909 
```




**Experiment 3**

The queries we tried above are similar to each other. Let's try something different that should touch all 3 tables:
```
"A query to list countries of birth, team names, last names and score year for highest scorers per year for each year after 1990"
```




In [ ]:
question="A query to list countries of birth, team names, last names and score year for highest scorers per year for each year after 1990"

query = build_query(secret_key, model, question)
print(query)

SELECT  birthCountry, name, lastName, year
FROM Players, Teams, Scoring
WHERE Players.playerID = Scoring.playerID
AND Teams.teamID = Scoring.teamID
AND Teams.year = Scoring.year
AND Scoring.year > 1990
AND Scoring.goals + Scoring.assists = (SELECT MAX(goals + assists)
                                       FROM Scoring
                                       WHERE Scoring.year = Teams.year)
ORDER BY year


In this case OpenAI forgets to qualify a "year" qualifier in SELECT row. But other than than that the query looks and performs corrrectly. Here's the output:

```
  BIRTHCOUNTRY         NAME         LASTNAME  YEAR  
 -------------- ------------------- --------- ----- 

 Canada         Pittsburgh Penguins Lemieux   1991  
 Canada         Pittsburgh Penguins Lemieux   1992  
 Canada         Los Angeles Kings   Gretzky   1993  
 Canada         Philadelphia Flyers Lindros   1994  
 Czech Republic Pittsburgh Penguins Jagr      1994  
 Canada         Pittsburgh Penguins Lemieux   1995  
 Canada         Pittsburgh Penguins Lemieux   1996  
 Czech Republic Pittsburgh Penguins Jagr      1997  
 Czech Republic Pittsburgh Penguins Jagr      1998  
 Czech Republic Pittsburgh Penguins Jagr      1999  
 Czech Republic Pittsburgh Penguins Jagr      2000  
 Canada         Calgary Flames      Iginla    2001  
 Sweden         Colorado Avalanche  Forsberg  2002  
 Canada         Tampa Bay Lightning St. Louis 2003  
 Czech Republic New York Rangers    Jagr      2005  
 Canada         Pittsburgh Penguins Crosby    2006  
 Russia         Washington Capitals Ovechkin  2007  
 Russia         Pittsburgh Penguins Malkin    2008  
 Sweden         Vancouver Canucks   Sedin     2009  
 Sweden         Vancouver Canucks   Sedin     2010  
 Russia         Pittsburgh Penguins Malkin    2011 
```



**Conclusion**

The GPT-3 engine performance is amazing. Not only it can reason about concepts in questions and model, but it can also construct complex SQL statements with different access strategies.

I have to confess that the examples above are the most interesting ones that I have tried. Sometime the engine misinterprets the text, sometime the SQL is not perfect and needs to be touched up. And sometime generated queries look good but don't do what have been asked.

Nevertheless, this is a giant step forward on the journey where we can clearly see that AI is here, it is incredibly powerful, and it will disrupt our SQL industry.
